In [13]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import copy
sys.path.append('/users/mtaranov/genome3D/')
from models_3d import Genome3D_DNN_FC_ResNet
from utils import normalize_features_sampl_by_f, concat_motifs,get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
contacts='pe'
path='/users/mtaranov/datasets_3d/by_chr_dist_matched_'+contacts+'/'
day='d0'
thres='10'

X_train_node1 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_train_window = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'


X_test_node1 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_test_window =path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_valid_window = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

y_train = get_labels(path+day+'_y_train_thres_'+thres+'.npy')
y_test = get_labels(path+day+'_y_test_thres_'+thres+'.npy')
y_valid = get_labels(path+day+'_y_valid_thres_'+thres+'.npy')

X_train=concat_motifs([X_train_node1, X_train_node2, X_train_window])
X_test=concat_motifs([X_test_node1, X_test_node2, X_test_window])
X_valid=concat_motifs([X_valid_node1, X_valid_node2, X_valid_window])

In [3]:
X_train.shape, X_test.shape, X_valid.shape

((6710, 1, 1920, 3), (988, 1, 1920, 3), (1796, 1, 1920, 3))

In [4]:
X_train_pairs = X_train.reshape(X_train.shape[0],X_train.shape[2]*X_train.shape[3])
X_valid_pairs = X_valid.reshape(X_valid.shape[0],X_valid.shape[2]*X_valid.shape[3])
X_test_pairs = X_test.reshape(X_test.shape[0],X_test.shape[2]*X_test.shape[3])

In [5]:
X_train_pairs.shape, X_test_pairs.shape, X_valid_pairs.shape

((6710, 5760), (988, 5760), (1796, 5760))

In [6]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features_sampl_by_f(X_train_pairs, X_valid_pairs, X_test_pairs)

In [14]:
dnn_normalized = Genome3D_DNN_FC_ResNet(num_features=5760, use_deep_CNN=False)
validation_data = (X_valid_normalized, y_valid)
dnn_normalized.train(X_train_normalized, y_train, validation_data)

Training model...
Train on 6710 samples, validate on 1796 samples
Epoch 1/100
6500/6710 [============================>.] - ETA: 0s - loss: 1.3414Epoch 0: validation loss: 0.705
Balanced Accuracy: 59.30%	auROC: 0.630	 auPRC: 0.594
Positve Accuracy: 0.11%	 Negative Accuracy: 59.24%
Recall at 5% | 10% | 25% | 50% FDR: 59.4% | 0.1% | 0.1% | 99.8%	Num Positives: 898	 Num Negatives: 898


6710/6710 [==============================] - 10s - loss: 1.3352 - val_loss: 0.7055
Epoch 2/100
6500/6710 [============================>.] - ETA: 0s - loss: 0.8817Epoch 1: validation loss: 0.679
Balanced Accuracy: 61.86%	auROC: 0.662	 auPRC: 0.638
Positve Accuracy: 0.45%	 Negative Accuracy: 61.47%
Recall at 5% | 10% | 25% | 50% FDR: 62.2% | 0.4% | 4.8% | 99.7%	Num Positives: 898	 Num Negatives: 898


6710/6710 [==============================] - 9s - loss: 0.8794 - val_loss: 0.6791
Epoch 3/100
6500/6710 [============================>.] - ETA: 0s - loss: 0.6679Epoch 2: validation loss: 0.652
Balanced Accuracy:

In [15]:
print(dnn_normalized.test(X_test_normalized, y_test))
print(dnn_normalized.test(X_train_normalized, y_train))
print(dnn_normalized.test(X_valid_normalized, y_valid))

Balanced Accuracy: 64.27%	auROC: 0.709	 auPRC: 0.694
Positve Accuracy: 0.00%	 Negative Accuracy: 72.67%
Recall at 5% | 10% | 25% | 50% FDR: 55.9% | 3.2% | 27.9% | 100.0%	Num Positives: 494	 Num Negatives: 494

Balanced Accuracy: 100.00%	auROC: 1.000	 auPRC: 1.000
Positve Accuracy: 100.00%	 Negative Accuracy: 100.00%
Recall at 5% | 10% | 25% | 50% FDR: 100.0% | 100.0% | 100.0% | 100.0%	Num Positives: 3355	 Num Negatives: 3355

Balanced Accuracy: 66.82%	auROC: 0.728	 auPRC: 0.697
Positve Accuracy: 0.67%	 Negative Accuracy: 72.72%
Recall at 5% | 10% | 25% | 50% FDR: 60.9% | 0.7% | 32.5% | 100.0%	Num Positives: 898	 Num Negatives: 898



In [17]:
dnn_normalized.save(contacts+'_dnn_FC_ResNet_seq_features_nodes_and_edges_'+'thres_'+thres, '/users/mtaranov/genome3D/')